# Welcome

Hi there! Welcome to the 1st IMG Data Science Hacakthon! As we stepping into a new Vanguard year, OKR is one of the most frequent topics has been talked about recently, so before we dive into the problem itself, I would like to walk you through the OKR of this Hackathon.

## Objective:
Build a model that helps predict Fund of Funds cashflows better than existing estimates

## Key Result -- Modeling Success:

The error of our prediction should be better than random or a simple constant term. Anything that improves on this baseline is a modeling success.

### Metric for modeling success key result: 

We are looking for people to come up with a model that takes all historical data including everything but today’s “actual” cashflow (and the “estimate_to_actual” value) and comes up with a better estimate of today’s estimate_to_actual ratio.

For each process_date, each portfolio we would like a better estimate that can be used on a go-forwards basis to better predict cashflows based on MSAE vs the observed “estimate to actual”)
## Key Result -- Learning Success:
Any approach we take should make intuitive sense, teach us something new about the data and be replicable on different samples.
### Metric for learning success: 
Documentation of the experimentation process that is easily understood and accepted by the business. Code that can be implemented by a FST into the rebalancing process. 



# Important note on the reality of working with a real life data science problem

We may find that this hackathon is unable to beat the benchmark; or it may highlight unanswerable questions about the quality or interpretation of the data. 

This is a real life data science problem at Vanguard and not a carefully crafted competition question! <b> So things may not go as planned - and that is OK! </b>

As long as we are iteratively experimenting, working towards the objective and sharing our findings, we are creating value with learning and the promotion of <b>NWoW</b>. 

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 500

# The Problem

Imagine you are a PM in EIG, when you are looking at our funds in the morning you may notice there is a difference in the actual amount of cash flow you received yesterday compared to what was projected to come in for the close (estimate cash amount), when you signed off at end of day yesterday. There are couple reasons why the cash flow is estimated and why the estimate is not always accurate:

1. NAVs for our funds are determined after the market close. We can't know the exact NAV prior to market close to calculate the cash flow of the fund for today, so we need to use yesterday's NAV to estimate. The market movement over the day can lead to valuation differences between the estimates (which is prior day's NAV) and the actuals, which get todays NAV.

2. There is certain activity from an estimate standpoint that can get processed after 4:00pm, and therefore, we would only see it on the actual cash flow files the next day.

3. There can be intraday processing corrections in the estimated cash figures but are offset and don't appear in the actual cash flow the next day.

4. Certain clients can submit their trades as late as 8am the next day and still get prior days NAV. This would bypass the estimate feed and only show up on the actual feed.

These differences in cash flows can result in our funds sitting on too much cash or being leveraged unexpectedly due to unknown client activity. For all funds, this can result in a few hundred million dollars of unexpected cash flow across the complex that the PMs were unaware of and thus were unable to invest on their benchmark date.

# The Opportunity


With regards to the concerns involved in the unexpected cash flow that PMs have to deal with, given the estimated cash flow you receive every day, can you give a prediction of the difference between the estimate and the actual cash flow for each fund every day? 

If we predict these differences, then we can take these predictions into account when positioning our funds at the end of each day, helping us become better positioned by reducing the leverage or cash next morning. 

Now that we understand the problem and a possible solution let's summarize the research question: Can we predict how much new cash flow we are expecting before market close 4pm ET and the next morning by using historical Fund of Funds cash flow data? Feel free to use other public data sources such as info from St. Louis Federal Reserve Data, which is provided in the makerlab environment.

# Datasets

In this hackathon, the main data you will be using to train your model is the "training_Hackathon_EstToAct.csv" file. Below, we will introduce each column and explain what each row represents for:

In [2]:
pd.read_csv('column_explanation.csv')

,Column Name in the data,Explanation
0,PORTFOLIO,The fund name
1,SCID,"The NeoXam ID of the fund, each fund has and only has one SCID"
2,SHARE_CLASS,"Individual class of a fund, for instance, a fund could have investor shares, admiral shares, etc."
3,PROCESS_DATE,"The date that the transaction was processed to the Transfer Agent(TA), which is also the date that PM needs to act on"
4,SOURCE,"Transfer Agent(TA) source system name, indicates whether the order is from RETAIL or INST"
5,CHANNEL,Detailed source channel
6,ESTIMATE,The estiamte of the cash flow
7,ACTUAL_AMOUNT,The actual cash flow
8,ESTIMATE_TO_ACTUAL,ACTUAL_AMOUNT-ESTIMATE


In [3]:
#pre-process and show the dataset
est_act_df = pd.read_csv('training_Hackathon_EstToAct.csv')
est_act_df['PROCESS_DATE'] = pd.to_datetime(est_act_df['PROCESS_DATE'])
est_act_df.head()

,PORTFOLIO,SCID,SHARE_CLASS,PROCESS_DATE,SOURCE,CHANNEL,ESTIMATE,ACTUAL_AMOUNT,ESTIMATE_TO_ACTUAL
0,Target Retirement 2030,SC81,695.0,2020-10-15,INST,VSTA,238.86,0.0,-238.86
1,Target Retirement 2030,SC81,695.0,2020-10-15,INST,VSTA,17779.68,0.0,-17779.68
2,Target Retirement 2030,SC81,695.0,2020-10-15,INST,VSTA,12317.10,0.0,-12317.10
3,Target Retirement 2030,SC81,695.0,2020-10-15,RETAIL,SMALL BUSINESS SERVICES,320489.13,0.0,-320489.13
4,Target Retirement 2030,SC81,695.0,2020-10-15,RETAIL,AUTOMATIC WITHDRAWAL PLAN,-33026.50,0.0,33026.50


### what does each row mean

Each row represents for fund (PORTFOLIO, SCID), on PROCESS_DATE, we have an aggregated ESTIMATE and ACTUAL_AMOUNT cashflow coming in from a (SOURCE, CHANNEL) that is excuted to a SHARE_CLASS of the fund. ESTIMATE_TO_ACTUAL = ACTUAL_AMOUNT - ESTIMATE.

For example, the first row of the above data shows that on 2020-10-15, for fund Target Retirement 2030, SCID SC81, share class 695, there is an estimate of cash flow 238.86 coming from source INST and channel VSTA, but the actual cashflow is 0, so ESTIMATE_TO_ACTUAL is ACTUAL_AMOUNT - ESTIMATE = -238.86.

## Data granularity

Even though the data we provide is pretty granular, as a PM, the estimate and actual cashflow you will see at the end of the day and at the beginning of the day is the aggregated data.

For a fund,for example, for fund that the PORTFOLIO = "Target Retirement 2020", SCID=AR74, Process_Date = 2019-01-16, there will be multiple observations in the dataset since you have different share classes / incoming source / trade date and settled date; <b>but as a PM, you only care about the total estimated amount vs the total actual amount for the fund. </b>

The two cells below will show you what is in the data and what a PM needs to see:

In [4]:
# What is in the dataset for AR74 on 2019-01-16 
est_act_df[(est_act_df['SCID']=='AR74')&(est_act_df['PROCESS_DATE']=='2019-01-16')]

,PORTFOLIO,SCID,SHARE_CLASS,PROCESS_DATE,SOURCE,CHANNEL,ESTIMATE,ACTUAL_AMOUNT,ESTIMATE_TO_ACTUAL
791827,Target Retirement 2020,AR74,682.0,2019-01-16,RETAIL,RETAIL WEB/WAVE,-5.424745e+05,0.000000e+00,5.424745e+05
791828,Target Retirement 2020,AR74,682.0,2019-01-16,INST,VSTA,1.843491e+04,0.000000e+00,-1.843491e+04
791829,Target Retirement 2020,AR74,682.0,2019-01-16,INST,VSTA,1.758770e+03,0.000000e+00,-1.758770e+03
791830,Target Retirement 2020,AR74,682.0,2019-01-16,INST,VSTA,4.528680e+03,0.000000e+00,-4.528680e+03
791831,Target Retirement 2020,AR74,682.0,2019-01-16,INST,VSTA,2.558520e+03,0.000000e+00,-2.558520e+03
791832,Target Retirement 2020,AR74,682.0,2019-01-16,RETAIL,AUTOMATIC WITHDRAWAL PLAN,-9.568966e+04,0.000000e+00,9.568966e+04
791833,Target Retirement 2020,AR74,682.0,2019-01-16,RETAIL,FUND/SERV (INCLUDES DCC&S),0.000000e+00,6.006622e+06,6.006622e+06
791834,Target Retirement 2020,AR74,682.0,2019-01-16,INST,VSTA,3.349240e+03,0.000000e+00,-3.349240e+03
791835,Target Retirement 2020,AR74,682.0,2019-01-16,RETAIL,FUND/SERV (INCLUDES DCC&S),0.000000e+00,-4.572000e+02,-4.572000e+02
791836,Target Retirement 2020,AR74,682.0,2019-01-16,RETAIL,VISTA TO VAST ROLLOVERS,0.000000e+00,4.304082e+05,4.304082e+05


As you can see from the dataset there are 37 records for Target Retirement 2020	on 2019-01-16, but below is what PM needs to know

In [5]:
# What is the estimate and actual for AR74 that PM seeing at the end of market on 2019-01-16 and the next morning
est_act_df[(est_act_df['SCID']=='AR74')&(est_act_df['PROCESS_DATE']=='2019-01-16')]\
.groupby(['PORTFOLIO', 'SCID','PROCESS_DATE'])['ESTIMATE', 'ACTUAL_AMOUNT', 'ESTIMATE_TO_ACTUAL'].sum()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  app.launch_new_instance()


,,,ESTIMATE,ACTUAL_AMOUNT,ESTIMATE_TO_ACTUAL
PORTFOLIO,SCID,PROCESS_DATE,,,
Target Retirement 2020,AR74,2019-01-16,-50667.62435,5.366985e+06,5.417653e+06


<b>On this date, the estiamate of cashflow is -50667.62435, but the actual amount is 5.366985e+06, so actual - estimate  = 5.417653e+06</b>

# Goal

As you have seen above, even though we provide granular data, your goal is helping PMs predict the difference between the actual and the estimated cashflow of a fund on a certain date.

To test your model, we picked 9065 combinations of (PROCESS_DATE, SCID, PORTFOLIO) that do not show up in the training data and saved them to a file called "result_template.csv".

<b>Your task is to fill "Your Prediction" in each row for the column called "ESTIMATE_TO_ACTUAL" in the file "result_template.csv"</b>

To predict those numbers, you may need to input the data "testing_Hackathon_EstToAct.csv" to your model and generate the prediction.

In "testing_Hackathon_EstToAct.csv", you have all the columns except for ACTUAL_AMOUNT and ESTIMATE_TO_ACTUAL for (PROCESS_DATE, SCID, PORTFOLIO) in "result_template.csv"



In [6]:
result_template = pd.read_csv('result_template.csv')
result_template.head(3)

,PROCESS_DATE,SCID,PORTFOLIO,ESTIMATE_TO_ACTUAL
0,2019-11-21,IS98,VVIF - Total Stock Market Index,Your Prediction
1,2019-08-08,WQ15,LifeStrategy Growth,Your Prediction
2,2020-07-23,FO91,VCEP Growth Select,Your Prediction


explain the test set more detailedly

In [7]:
#pre-process and show the testing set
test_df = pd.read_csv('testing_Hackathon_EstToAct.csv')
test_df['PROCESS_DATE'] = pd.to_datetime(test_df['PROCESS_DATE'])
test_df.head()

,PORTFOLIO,SCID,SHARE_CLASS,PROCESS_DATE,SOURCE,CHANNEL,ESTIMATE
0,Target Retirement 2045 Trust,BJ34,1469.0,2020-10-15,INST,VSTA,55.59
1,Target Retirement 2045 Trust,BJ34,1658.0,2020-10-15,INST,VSTA,551329.22
2,Target Retirement 2045 Trust,BJ34,7742.0,2020-10-15,INST,WEBT,57815.42
3,Target Retirement 2045 Trust,BJ34,1681.0,2020-10-15,INST,USER,-344289.49
4,Target Retirement 2045 Trust,BJ34,1469.0,2020-10-15,INST,VSTA,4478029.66


# Submission

There <b>two things</b> to be submitted:

1. Test set prediction result: 
    In this hackathon, as mentioned above, you only need to fill "Your Prediction" in each row for "ESTIMATE_TO_ACTUAL" in the file "result_template.csv" and please save the dataframe to a local file called "result.csv"
    Below this sell, I give an example that fills the ESTIMATE_TO_ACTUAL column with all zeros and saves it to "result.csv" file
2. One notebook:
    Please also submit your notebook to introduce your thoughts and your model
    
After finishing hackathon, <b> Please notify Chuqi Yang that you have finished the hackathon and which one is your test set prediction result and notebook</b>




In [8]:
#Load template
result = pd.read_csv('result_template.csv')
result['ESTIMATE_TO_ACTUAL'] = 0

In [9]:
#what's in the result
result.head()

,PROCESS_DATE,SCID,PORTFOLIO,ESTIMATE_TO_ACTUAL
0,2019-11-21,IS98,VVIF - Total Stock Market Index,0
1,2019-08-08,WQ15,LifeStrategy Growth,0
2,2020-07-23,FO91,VCEP Growth Select,0
3,2019-06-25,MK83,VCEP Total Bond Select,0
4,2020-06-19,ZD97,VCEP Total International Bond Pool Select,0


In [10]:
# save the result to local
result.to_csv('result.csv')

# Metrics

In this hackathon, we will use [MAE](https://en.wikipedia.org/wiki/Mean_absolute_error)__(__MAE__) to measure your model's performance quantitatively. The equation of MAE is:
\begin{equation*}
MAE   = \frac{\sum_{i=1}^{N}|y-y_{pred}|}{N}
\end{equation*}

so it's measuring the average difference between your prediction $y_{pred}$ and real $y$, from $0$ to $\infty$, the smaller MAE, the better prediction!



After the deadline of submission(Nov 15th 11:59pm ET), we will collect your team's result and test your MAE. However, since it's a real vanguard business-related hackathon, we will <b>not only judge the result by your MAE but also work with our business owner from EIG to determine whether the solution is helpful in EIG's real business</b>

# Awards

We will release the MAE ranking but there will be <b>NO awards</b> for this hackathon. But please remember, this is a <b>vanguard business-related problem</b>, so we want to invite the teams(persons) who made a good prediction and/or good model to present their models and thoughts in the ceremony event; furthermore, if the model is really promising, we may work with you to improve the model and potentially be adapted by PMs in EIG! 



# Rules
    1. Do not download or dispense the data. 
    
    2. External data/packages are allowed. You can install your package by PIP.
    
    3. Due to the capacity, the competition is only open to crews from IMG(including rotational crew) and IMG-related departments(PRD, GIFS, IOPs), please let us know if you are from other department by emailing chuqi_yang2@vanguard.com .

    4. The hackathon named above is a skills-based competition to promote and further the field of data science in investment. 

    5. Your competition submissions must conform to the requirements mentioned above.

    6. The hackathon period is from Nov 9th - Nov 15th, any submission after Nov 15th 11:59 PM ET will not be counted. 

    7. We ask that you respect the spirit of the hackathon and do not cheat. 

# Supplment data

In this Hackathon, we encourage you to bring external data. Here, we provide you some examples, for example, S&P 500, 10-Year Treasury Constant Maturity Rate, VIX and Consumer Sentiment Index. By using economic data, you may or may not able to improve your model's result.

In [11]:
snp500 = pd.read_csv('SP500.csv')
snp500.head(1)

,DATE,SP500
0,2019-01-02,2510.03


In [12]:
dgs = pd.read_csv('DGS10.csv')
dgs.head(1)

,DATE,DGS10
0,2019-01-02,2.66


CBOE Volatility Index: VIX

In [13]:
vix = pd.read_csv('VIXCLS.csv')
vix.head(1)

,DATE,VIXCLS
0,2019-01-02,23.22


University of Michigan: Consumer Sentiment

In [14]:
umscent_df = pd.read_csv('UMCSENT.csv')
umscent_df.head(1)

,DATE,UMCSENT
0,2019-01-01,91.2
